# Mnist Classification using Tensorflow

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
input_size = 784
output_size = 10
hiddenlayer_size = 200
tf.reset_default_graph()
# Build NN
inputs = tf.placeholder(tf.float32, [None, input_size])
targets = tf.placeholder(tf.float32, [None, output_size])

weight1 = tf.get_variable("weight1", [input_size, hiddenlayer_size])
biases1 = tf.get_variable("biases1", [hiddenlayer_size])
output1 = tf.nn.relu(tf.matmul(inputs,weight1) + biases1)

weight2 = tf.get_variable("weight2", [hiddenlayer_size, hiddenlayer_size])
biases2 = tf.get_variable("biases2", [hiddenlayer_size])
output2 = tf.nn.relu(tf.matmul(output1, weight2) + biases2)

weight3 = tf.get_variable("weight3", [hiddenlayer_size, output_size])
biases3 = tf.get_variable("biases3", [output_size])
outputs = tf.matmul(output2,weight3) + biases3

# define loss function
loss = tf.nn.softmax_cross_entropy_with_logits(logits = outputs, labels = targets)
mean_loss = tf.reduce_mean(loss)

# optimizer
optimize = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(mean_loss)

#pridiction accuracy using argmax
eq_target = tf.equal(tf.argmax(outputs,1), tf.argmax(targets,1))
accuracy = tf.reduce_mean(tf.cast(eq_target, tf.float32))

#session preperation
sess = tf.InteractiveSession()
# variable initialization
initializer = tf.global_variables_initializer()
sess.run(initializer)

#batching
batch_size = 200
batch_number = mnist.train._num_examples // batch_size

#setup early stopping
epochs = 15
pre_validation_loss = 9999999.

for epoch_counter in range(epochs):
    epoch_loss = 0.
    for batch_counter in range(batch_number):
        input_batch, target_batch = mnist.train.next_batch(batch_size)
        _, batch_loss = sess.run([optimize, mean_loss], feed_dict = {inputs: input_batch, targets: target_batch})
        epoch_loss += batch_loss
    
    #mean training loss
    epoch_loss /= batch_number
    input_batch, target_batch = mnist.validation.next_batch(mnist.validation._num_examples)
    validation_loss, validation_accuracy = sess.run([mean_loss,accuracy],feed_dict={inputs: input_batch, targets: target_batch})
    
    print "epoch "+ str(epoch_counter+1) + ".Training loss: "+'{0:.3f}'.format(epoch_loss)+".Validation loss: " + '{0:.3f}'.format(validation_loss)+".Validation Accuracy: "+'{0:.2f}'.format(validation_accuracy*100.)+'%'
    if validation_loss > pre_validation_loss:
        break
    pre_validation_loss = validation_loss
    


In [ ]:
#test model
input_batch, target_batch = mnist.test.next_batch(mnist.test._num_examples)
test_accuracy = sess.run([accuracy], 
    feed_dict={inputs: input_batch, targets: target_batch})

# print (test_accuracy)
test_accuracy_percent = test_accuracy[0] * 100.

# Print the test accuracy formatted in percentages
print('Test accuracy: '+'{0:.2f}'.format(test_accuracy_percent)+'%')

In [ ]:
# final output by running prediction on test data set
for x in range(100):
    x_train = mnist.test.images[x,:].reshape(1,784)
    y_train = mnist.test.labels[x,:]
    label = y_train.argmax()
    prediction = sess.run(outputs, feed_dict={inputs: x_train}).argmax()
    if prediction!=label:
        plt.title('prediction: %d label: %d' %(prediction, label))
        plt.imshow(x_train.reshape([28,28]), cmap=plt.get_cmap('gray_r'))
        plt.show()